# NFL Play-by-Play Data Analysis

### Assist from https://towardsdatascience.com/analyzing-and-plotting-nfl-data-with-nflfastpy-and-plotly-a170a09cad6

In [1]:
import nfl_data_py as nfl
import pandas as pd
import plotly.graph_objects as go

In [2]:
df_2021 = nfl.import_pbp_data([2021])

2021 done.
Downcasting floats.


In [3]:
cols = list(nfl.see_pbp_cols())

In [4]:
cols

['play_id',
 'game_id',
 'old_game_id',
 'home_team',
 'away_team',
 'season_type',
 'week',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'quarter_end',
 'drive',
 'sp',
 'qtr',
 'down',
 'goal_to_go',
 'time',
 'yrdln',
 'ydstogo',
 'ydsnet',
 'desc',
 'play_type',
 'yards_gained',
 'shotgun',
 'no_huddle',
 'qb_dropback',
 'qb_kneel',
 'qb_spike',
 'qb_scramble',
 'pass_length',
 'pass_location',
 'air_yards',
 'yards_after_catch',
 'run_location',
 'run_gap',
 'field_goal_result',
 'kick_distance',
 'extra_point_result',
 'two_point_conv_result',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'timeout',
 'timeout_team',
 'td_team',
 'td_player_name',
 'td_player_id',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_differential',
 'po

In [5]:
# import rosters for future join
rosters = nfl.import_rosters([2021])

In [6]:
# import teams (with colors) for future join
teams = nfl.import_team_desc()

In [7]:
# filter to regular season
df_2021 = df_2021[df_2021["season_type"] == "REG"]

# remove two point attempts
df_2021 = df_2021[df_2021["two_point_attempt"] == False]

# filter to pass plays
df_2021 = df_2021[df_2021["play_type"] == "pass"]

In [8]:
df_2021.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
3,76.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0.0,1.0,0.032412,1.165133,5.803177,4.0,0.896654,0.125098,0.697346,30.265415
4,100.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0.0,1.0,-1.532898,0.256036,4.147637,2.0,0.965009,0.965009,0.978253,2.174652
6,152.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,ARI,away,TEN,...,1.0,1.0,2.692890,0.567838,7.420427,4.0,1.000000,1.000000,0.458989,54.101131
8,218.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,ARI,away,TEN,...,1.0,1.0,-0.511090,1.036891,10.339405,9.0,0.478471,0.079696,0.684949,31.505138
9,253.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,ARI,away,TEN,...,1.0,1.0,2.182015,0.517965,3.045047,1.0,1.000000,0.998799,0.775463,22.453720


In [9]:
# join with the roster table to get player names
df_2021 = df_2021.merge(rosters[["player_name", "player_id"]], left_on="receiver_player_id", right_on="player_id")

In [10]:
df_2021["player_name"].unique()

array(['Derrick Henry', 'Chester Rogers', 'DeAndre Hopkins',
       'Chase Edmonds', 'A.J. Green', 'Christian Kirk', 'Rondale Moore',
       'Sam Cooper', 'Mark Vital', 'Demetrius Harris', 'Jeremy McNichols',
       'Amani Hooker', 'A.J. Brown', 'Anthony Firkser', 'Julio Jones',
       'Geoff Swaim', 'Nick Westbrook-Ikhine', 'Maxx Williams',
       'Darren Waller', 'Alec Ingold', 'Hunter Renfrow', 'Josh Jacobs',
       "Ty'Son Williams", 'Sammy Watkins', 'Mark Andrews',
       'Marquise Brown', 'Devin Duvernay', 'Josh Oliver', 'Kenyan Drake',
       'Zay Jones', 'Bryan Edwards', 'Patrick Ricard', 'Willie Snead',
       'Darnell Mooney', 'Marquise Goodwin', 'Tyler Higbee',
       'Van Jefferson', 'Allen Robinson', 'Cole Kmet', 'Damien Williams',
       'Cooper Kupp', 'DeSean Jackson', 'Damiere Byrd',
       'Darrell Henderson', 'Robert Woods', 'David Montgomery',
       'Jimmy Graham', 'Nick Chubb', 'Austin Hooper', 'Andy Janovich',
       'Anthony Schwartz', 'David Njoku', 'Travis Kelc

In [11]:
# join with team table to get team colors
df_2021 = df_2021.merge(teams[["team_abbr", "team_color"]], left_on="posteam", right_on="team_abbr")

In [12]:
# get total yards and touchdowns by week
df_agg = (
    df_2021.groupby(["player_name", "team_abbr", "team_color", "week"], as_index=False)
    .agg({"receiving_yards": "sum", "touchdown": "sum"})
)

In [13]:
# look at Stefon Diggs only
df_agg[df_agg["player_name"] == "Stefon Diggs"]

,player_name,team_abbr,team_color,week,receiving_yards,touchdown
4877,Stefon Diggs,BUF,#00338D,1,69.0,0.0
4878,Stefon Diggs,BUF,#00338D,2,60.0,1.0
4879,Stefon Diggs,BUF,#00338D,3,62.0,0.0
4880,Stefon Diggs,BUF,#00338D,4,114.0,0.0
4881,Stefon Diggs,BUF,#00338D,5,69.0,0.0
4882,Stefon Diggs,BUF,#00338D,6,89.0,1.0
4883,Stefon Diggs,BUF,#00338D,8,40.0,1.0
4884,Stefon Diggs,BUF,#00338D,9,85.0,0.0
4885,Stefon Diggs,BUF,#00338D,10,162.0,1.0
4886,Stefon Diggs,BUF,#00338D,11,23.0,2.0


In [17]:
fig = go.Figure()
for name, values in df_agg.groupby("player_name"):
    if values["receiving_yards"].sum() > 800:
        fig.add_trace(
            go.Scatter(
                x=values["week"], 
                y=values["receiving_yards"].cumsum(), 
                name=name, 
                mode="markers+lines", 
                line_color=values.iloc[0].team_color,
                hovertemplate=f"<b>{name}</b><br>%{{y}} yds through week %{{x}}<extra></extra>"
            )
        )
    
fig.update_layout(
    font_family="Averta, sans-serif",
    hoverlabel_font_family="Averta, sans-serif",
    xaxis_title_text="Week",
    xaxis_title_font_size=18,
    xaxis_tickfont_size=16,
    yaxis_title_text="Receiving Yards",
    yaxis_title_font_size=18,
    yaxis_tickfont_size=16,
    hoverlabel_font_size=16,
    legend_font_size=16,
    height=1000,
    width=1000
)
    
fig.show()

In [21]:
fig = go.Figure()
for name, values in df_agg.groupby("player_name"):
    if values["touchdown"].sum() > 6:
        fig.add_trace(
            go.Scatter(
                x=values["week"], 
                y=values["touchdown"].cumsum(), 
                name=name, 
                mode="markers+lines", 
                line_color=values.iloc[0].team_color,
                hovertemplate=f"<b>{name}</b><br>%{{y}} rec TDs through week %{{x}}<extra></extra>"
            )
        )
    
fig.update_layout(
    font_family="Averta, sans-serif",
    hoverlabel_font_family="Averta, sans-serif",
    xaxis_title_text="Week",
    xaxis_title_font_size=18,
    xaxis_tickfont_size=16,
    yaxis_title_text="Receiving TDs",
    yaxis_title_font_size=18,
    yaxis_tickfont_size=16,
    hoverlabel_font_size=16,
    legend_font_size=16,
    height=1000,
    width=1000
)
    
fig.show()